### 全局配置

In [1]:
from dataclasses import dataclass
from typing import Optional
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# 全局配置类
@dataclass
class ModelArgs:
    dim: int = 4096  # llama嵌入维度为4096
    n_layers: int = 32
    n_heads: int = 32 # Q的头数
    n_kv_heads: Optional[int] = None # K,V的头数 使用Group Multiple Query
    vocab_size: int = -1
    multiple_of: int = 256
    ffn_dim_multiplier: Optional[float] = None
    norm_eps: float = 1e-5

    # KV cache变量
    max_batch_size: int = 32
    max_seq_len: int = 2048

    device: str = None

## RoPE
### 计算旋转矩阵 $R_{\Theta, m}^d$
实际上需要进行计算简化，因此函数计算得到的是一个复数矩阵

<img src="imgs/complexMat.png" alt="mT@thetha" style="width:30%; height:auto;" />


In [ ]:
def precompute_theta_pos_frequencies(head_dim: int,seq_len: int,device: str, theta: float = 10000.0):
    assert head_dim % 2 == 0, "单头嵌入维度必须能被2整除"

    # (Head_dim / 2)  [0,2,4,6,...Head_dim - 2]
    theta_numerator = torch.arange(0, head_dim, 2).float()

    # (Head_dim / 2)  θi公式
    theta = 1.0 / (theta ** (theta_numerator / head_dim)).to(device)

    # (Seq_len)       [0,1,2,3,4,....,Seq_len - 1]
    m = torch.arange(seq_len,device=device)

    # m.T @ theta (注意仅为示例,m与θ为1D向量)
    # (Seq_len,1) @ (1,Head_dim / 2) => (Seq_len, Head_dim / 2)
    freqs = torch.outer(m,theta).float()

    # 转换为极坐标形式 c = R * exp(m * theta), R = 1 
    # torch.polar(abs = 1,angle = freqs)
    freqs_complex = torch.polar(torch.ones_like(freqs), freqs)
    
    return freqs_complex


### 计算旋转过程$R_{\Theta, m}^d x$
简化旋转矩阵过程后的最终形式:

<img src="imgs/Rx.png" alt="Rx" style="width:70%; height:auto;" />



In [ ]:
def apply_rotary_embeddings(x: torch.Tensor, freqs_complex: torch.Tensor, device: str):

    # STEP1,2: 每两个合并为1个复数
    # (B, Seq_len , H, Head_dim) -> (B, Seq_len, H, Head_dim/2, 2) 
    # -> 合并复数后: (B, Seq_len , H, Head_dim)
    x_complex = torch.view_as_complex(x.float().reshape(*x.shape[:-1], -1, 2))

    # 确保freqs_complex维度匹配
    # (Seq_len, Head_dim/2)    ->  (1, Seq_len, Head_dim/2)
    # (1, Seq_len, Head_dim/2) ->  (1, Seq_len, 1, Head_dim/2)
    freqs_complex = freqs_complex.unsqueeze(0).unsqueeze(2)

    # STEP3 相乘
    # (B, Seq_len, H, Head_dim/2) * (1, Seq_len, 1, Head_dim/2)
    # => (B, Seq_len, H, Head_dim/2)
    x_rotated = x_complex * freqs_complex

    # STEP4 转换为数对
    # (B, Seq_len, H, Head_dim/2) -> (B, Seq_len, H, Head_dim/2, 2)
    x_out = torch.view_as_real(x_rotated)

    # STEP5 Flatten
    # (B, Seq_len, H, Head_dim/2, 2) -> (B, Seq_len, H, Head_dim/2) 
    x_out = x_out.reshape(*x.shape)

    return x_out.type_as(x).to(device)

## Attention
### KV-cache(推理)
* Q仅为一个Token
* 每次计算出新的K,V则加入cache
* 得到一个y

<img src="imgs/kv-cache.PNG" alt="kv-cache" style="width:40%; height:auto;" />

### Group multiquery attention
* 兼顾性能与效率
* Q头数不变,KV头数变少

<img src="imgs/GMHA.PNG" alt="GMHA" style="width:40%; height:auto;" />

### K,V RoPE
* 从K,V角度修改attention函数

<img src="imgs/RoPE.PNG" alt="RoPE" style="width:40%; height:auto;" />

In [ ]:
def repeat_kv(x: torch.Tensor, n_rep: int) -> torch.Tensor:
    batch_size, seq_len, n_kv_heads, head_dim = x.shape
    if n_rep == 1:
        return x
    return(
        # (B, Seq_len, N_KV_heads, 1, Head_Dim)
        x[:, :, :, None, :]
        # (B, Seq_len, N_KV_heads, N_Rep , 1, Head_Dim)
        .expand(batch_size, seq_len, n_kv_heads, n_rep, head_dim)
        # (B, Seq_len, N_KV_heads * N_Rep , 1, Head_Dim)
        .reshape(batch_size, seq_len, n_kv_heads * n_rep, head_dim)
        # N_KV_heads * N_Rep = N_Q_heads 变为MHA
    )


class SelfAttention(nn.Module):
    def __init__(self,args: ModelArgs):
        super().__init__()

        # 分组查询机制:Q与KV头的数量不同
        self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads
        self.n_heads_q  = args.n_heads
        # 得到的比值指导KV头的复制
        self.n_rep = self.n_heads_q // self.n_kv_heads
        # 仅头的数量不同,所有嵌入维度一致!  4096/32
        self.head_dim = args.dim // args.n_heads

        self.wq = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        self.wk = nn.Linear(args.dim, self.n_kv_heads * self.head_dim, bias=False)
        self.wv = nn.Linear(args.dim, self.n_kv_heads * self.head_dim, bias=False)
        self.wo = nn.Linear(args.n_heads * self.head_dim, args.dim, bias=False)

        # kv-cache
        # 大小: (B, Seq_len, H_kv, Head_dim)
        self.cache_k = torch.zeros((args.max_batch_size,args.max_seq_len,self.n_kv_heads,self.head_dim))
        self.cache_v = torch.zeros((args.max_batch_size,args.max_seq_len,self.n_kv_heads,self.head_dim))

    def forward(
        self,
        x: torch.Tensor, # (B, 1, dim) 仅一行,使用KV-cache
        start_pos: int,
        freqs_complex: torch.Tensor
    ):
        # (B, 1, dim)
        batch_size, seq_len, _ = x.shape

        # (B, 1, dim) -> (B, 1, H_Q * Head_dim)
        xq = self.wq(x)
        # (B, 1, dim) -> (B, 1, H_KV * Head_dim)
        xk = self.wk(x)
        # (B, 1, dim) -> (B, 1, H_KV * Head_dim)
        xv = self.wv(x)

        # 都只有一行
        # (B, 1, H_Q * Head_dim) -> (B, 1, H_Q , Head_dim)
        xq = xq.view(batch_size, seq_len, self.n_heads_q, self.head_dim)
        # (B, 1, H_KV * Head_dim) ->  (B, 1, H_KV , Head_dim)
        xk = xk.view(batch_size, seq_len, self.n_kv_heads, self.head_dim)
        # (B, 1, H_KV * Head_dim) ->  (B, 1, H_KV , Head_dim)
        xv = xv.view(batch_size, seq_len, self.n_kv_heads, self.head_dim)

        # RoPE 给q,k做旋转
        xq = apply_rotary_embeddings(xq,freqs_complex,device=x.device)
        xk = apply_rotary_embeddings(xk,freqs_complex,device=x.device)

        # 更新kv-cache
        # (B, 1, H_KV , Head_dim) 加入==> (B, Seq_len, H_kv, Head_dim)
        self.cache_k[:batch_size, start_pos : start_pos + seq_len] = xk
        self.cache_v[:batch_size, start_pos : start_pos + seq_len] = xv

        # 更新后取出KV矩阵 (B, Seq_len_now, H_kv, head_dim)
        keys   = self.cache_k[:batch_size, : start_pos + seq_len]
        values = self.cache_v[:batch_size, : start_pos + seq_len]

        # 实现分组查询的简单方式,根据倍率复制KV权重再按标准MHA计算
        #  (B, Seq_len_now, H_kv, head_dim) ->  (B, Seq_len_now, H_q, head_dim)
        keys = repeat_kv(keys,self.n_rep)
        values = repeat_kv(values,self.n_rep)

        # 准备进行计算
        # (B, 1, H_q, head_dim) -> (B, H_q, 1, head_dim)
        xq = xq.reshape(1,2)
        # (B, Seq_len_now , H_q, head_dim) -> (B, H_q, Seq_len_now, head_dim)
        keys = keys.reshape(1,2)
        values = values.reshape(1,2)

        # (B, H_q, 1, head_dim) @ (B, H_q, head_dim, Seq_len_now) = (B, H_q, 1, Seq_len_now)
        scores = torch.matmul(xq,keys.transpose(2,3)) / math.sqrt(self.head_dim)
        scores = F.softmax(scores.float(),dim = -1).type_as(xq)
        # (B, H_q, 1, Seq_len_now) @ (B, H_q, Seq_len_now, head_dim) =  (B, H_q, 1, head_dim)
        output = torch.matmul(scores,values)
        # (B, H_q, 1, head_dim) -> (B, 1, H_q, head_dim) -> (B, 1, H_q*head_dim) = (B, 1, Dim)
        output = (output.transpose(1, 2).contiguous().view(batch_size,seq_len,-1))

        return self.wo(output)

### RMSNorm

$
y = \frac{x}{\mathrm{RMS}(x)} \cdot g
$

其中

$
\mathrm{RMS}(x) = \sqrt{\frac{1}{n} \sum_{i=1}^n x_i^2 + \epsilon}
$

* $x \in \mathbb{R}^n$：输入向量（一个 token 的隐藏层表示）
* $g \in \mathbb{R}^n$：可学习的缩放系数（类似 LayerNorm 的 $\gamma$）
* $\epsilon$：防止除零的稳定项
* $y$：归一化后的输出

In [ ]:
class RMSNorm(nn.Module):
    def __init__(self,dim: int,eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        # 缩放参数g,可学习
        self.weight = nn.Parameter(torch.ones(dim))

    def _norm(self, x: torch.Tensor):
        # (B, Seq_len, dim) * (B, Seq_len, 1) = (B, Seq_len, dim)
        # rsqrt(x) = 1 / sqrt(x)
        return x * torch.rsqrt(x.pow(2).mean(-1,keepdim=True) + self.eps)
    
    def forward(self, x: torch.Tensor):
        # (Dim) * (B, Seq_len, Dim) = (B, Seq_len, Dim)
        return self.weight * self._norm(x.float()).type_as(x)

### Feedforward(SiLU)
<img src="imgs/silu.PNG" alt="model arch" style="width:30%; height:auto;" />

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()

        hidden_dim = 4 * args.dim
        hidden_dim = int(2 * hidden_dim / 3)

        # 该倍数用于对齐不同模型架构的隐藏层维度
        if args.ffn_dim_multiplier is not None:
            hidden_dim = int(args.ffn_dim_multiplier * hidden_dim)

        '''
        自动将维度对齐到指定multiple_of的倍数(向上进行最小对齐)
        hidden_dim = 7
        multiple_of = 5
        则公式= 5 * (7 + 5 - 1) / 5 = 10 现在是10的倍数了
        '''
        hidden_dim = args.multiple_of * ( (hidden_dim + args.multiple_of - 1) // args.multiple_of )

        # W部分 
        self.w1 = nn.Linear(args.dim,hidden_dim,bias=False)
        # W2部分
        self.w2 = nn.Linear(hidden_dim,args.dim,bias=False)
        # V部分
        self.w3 = nn.Linear(args.dim,hidden_dim,bias=False)

    def forward(self, x: torch.Tensor):
        # Swish1(xW)  (B,seq_len,dim) -> (B,seq_len,hidden_dim)
        swish = F.silu(self.w1(x))
        # xV          (B,seq_len,dim) -> (B,seq_len,hidden_dim)
        x_V = self.w3(x)
        # Swish1(xW) * xV
        x = swish * x_V
        # (Swish1(xW) * xV)W2
        x = self.w2(x)

        return x


### 总体模型结构
<img src="imgs/arch.PNG" alt="model arch" style="width:30%; height:auto;" />

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()

        self.n_heads = args.n_heads
        self.dim = args.dim
        self.head_dim = args.dim // args.n_heads

        self.attention_norm = RMSNorm(args.dim, eps=args.norm_eps)
        self.attention = SelfAttention(args)
        self.ffn_norm = RMSNorm(args.dim, eps=args.norm_eps)
        self.feed_forward = FeedForward(args)

    def forward(self, x: torch.Tensor, start_pos: int, freqs_complex: torch.Tensor):
        
        h = x + self.attention.forward(
            self.attention_norm(x), start_pos, freqs_complex
        )

        out = h + self.feed_forward.forward(self.ffn_norm(h))
        return out



class Transformer(nn.Module):

    def __init__(self,args: ModelArgs):
        super().__init__()

        assert args.vocab_size != -1, "未设置词表大小"

        self.args = args
        self.vocab_size = args.vocab_size
        self.n_layers = args.n_layers
        # 嵌入层
        self.tok_embeddings = nn.Embedding(self.vocab_size,args.dim)

        # N层堆叠的encoder块
        self.layers = nn.ModuleList()
        for layer_id in range(args.n_layers):
            self.layers.append(EncoderBlock(args))

        self.norm = RMSNorm(args.dim,eps = args.norm_eps)
        
        self.output = nn.Linear(args.dim, self.vocab_size, bias=False)

        # RoPE位置编码
        self.freqs_complex = precompute_theta_pos_frequencies(
            self.args.dim // self.args.n_heads,
            self.args.max_seq_len * 2,
            device = self.args.device
        )

    def forward(self, tokens: torch.Tensor, start_pos: int):
        # KV-cache仅限推理!

        # (B, Seq_len)
        batch_size, seq_len = tokens.shape
        assert seq_len == 1, "KV缓存,Q仅为每次更新的一个token 一次处理一个token!"

        # (B, Seq_len) -> (B, Seq_len , Dim)
        h = self.tok_embeddings(tokens)

        # RoPE编码
        freq_complex = self.freqs_complex[start_pos:start_pos + seq_len]

        for layer in self.layers:
            h = layer(h,start_pos,freq_complex)
        
        # RMSNorm
        h = self.norm(h)

        # Linear
        output = self.output(h).float()

        # Softmax 在 loss 中
        return output



### Token预测策略

| 方法        | 思路 | 优点 | 缺点 | 示例 |
|-------------|------|------|------|------|
| **Greedy Decoding** | 每一步都选概率最大的 token | 简单、速度快 | 容易陷入局部最优，缺乏多样性 | 分布 `[A:0.7, B:0.2, C:0.1]` → 选 `A` |
| **Beam Search** | 每一步保留 top-k 条候选路径，最后选概率最高的序列 | 结果比贪婪更优，考虑更多全局信息 | 计算量大，结果仍偏确定 | beam size=3 → 保留3条路径并扩展 |
| **Top-K** | 每一步只在前 K 个 token 中按概率采样 | 增加多样性，可控范围 | K 太小 → 多样性不足；K 太大 → 质量下降 | 分布 `[A:0.6, B:0.3, C:0.1, D:0.05]`，K=3 → 从 `[A,B,C]` 中采样 |
| **Top-P** | 选择累计概率 ≥ p 的最小集合，然后采样 | 自适应候选数，质量较好，多样性强 | 仍有随机性，结果不稳定 | 分布 `[A:0.6, B:0.25, C:0.1, D:0.05]`，p=0.9 → 选 `[A,B,C]` 中采样 |

### LLM 推理中的 Logit 与 Temperature

| 概念 | 定义 | 公式 | 作用 | 示例 |
|------|------|------|------|------|
| **Logit** | 模型输出层 softmax 之前的原始分数，长度 = 词表大小 | $$ p_i = \frac{\exp(z_i)}{\sum_j \exp(z_j)} $$ | 表示模型对每个 token 的相对偏好，数值大 → 更可能被选中 | logits = [2.0, 1.0, 0.1] → softmax 概率 = [0.65, 0.24, 0.11] |
| **Temperature** | softmax 前对 logit 进行缩放的系数 \(T\) | $$ p_i = \frac{\exp(z_i / T)}{\sum_j \exp(z_j / T)} $$ | 控制分布的“尖锐”程度：<br> - T < 1 → 更确定<br> - T > 1 → 更随机<br> - T → 0 → 贪婪解码<br> - T → ∞ → 近似均匀分布 | logits = [2.0, 1.0, 0.1]：<br> - T=1 → [0.65, 0.24, 0.11]<br> - T=0.5 → [0.84, 0.13, 0.03]<br> - T=2.0 → [0.45, 0.33, 0.22] |

### 🦙 Llama2采用Top-P + Temperature

In [ ]:
from typing import Optional
import time
from pathlib import Path
import json
from sentencepiece import SentencePieceProcessor
from tqdm import tqdm

class LLaMA:
    def __init__(
        self,
        model: Transformer,
        tokenizer: SentencePieceProcessor,
        model_args: ModelArgs
    ):
        self.model = model
        self.tokenizer = tokenizer
        self.args = model_args

    # 读取权重文件llama-2-7b
    @staticmethod
    def build(
        checkpoints_dir: str,
        tokenizer_path: str,
        load_model: bool,
        max_seq_len: int,
        max_batch_size: int,
        device: str
    ):
        # 计时
        prev_time = time.time()

        # 加载权重文件
        if load_model:
            checkpoints = sorted(Path(checkpoints_dir).glob("*.pth"))
            assert len(checkpoints) > 0, f"{checkpoints_dir} 路径下未发现权重文件"
            ckpt_path = checkpoints[0]
            print(f"加载权重文件 {ckpt_path}")
            # 得到权重字典
            checkpoint = torch.load(ckpt_path,map_location='cpu')

            print(f"加载权重花费时间: {time.time() - prev_time:.2f}s")
            prev_time = time.time()

        # 加载配置文件
        with open(Path(checkpoints_dir) / "params.json", "r") as f:
            params = json.loads(f.read())
            
        model_args: ModelArgs = ModelArgs(
            max_seq_len=max_seq_len,
            max_batch_size=max_batch_size,
            device=device,
            **params  #传入其他配置自动匹配对应项
        )

        # 加载分词器
        tokenizer = SentencePieceProcessor()
        tokenizer.load(tokenizer_path)
        model_args.vocab_size = tokenizer.vocab_size()

        # 半精度计算
        if device == "cuda":
            torch.set_default_tensor_type(torch.cuda.HalfTensor)
        else:
            torch.set_default_tensor_type(torch.BFloat16Tensor)

        # ⚠️Pytorch高版本上面的代码会报出警告
        # if device == "cuda":
        #     torch.set_default_dtype(torch.float16)
        #     torch.set_default_device("cuda")
        # else:
        #     torch.set_default_dtype(torch.bfloat16)
        #     torch.set_default_device("cpu")

        print(f"将模型部署至设备: {device}")
        model = Transformer(model_args).to(device)
        print(f"完成将模型部署至设备")

        # 真正地逐Key匹配加载权重
        if load_model:
            # 删除旋转位置编码数据,我们直接计算
            # 权重是字典
            
            # del checkpoint['rope.freqs']
            
            print(f"加载权重字典中")
            model.load_state_dict(checkpoint,strict=True)
            print(f"加载了权重字典,耗时{time.time() - prev_time:.2f}s")

        return LLaMA(model,tokenizer,model_args)
    

